In [2]:
!pip install transformers datasets
!pip install timm

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 52, in main
    servic

In [3]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceM4/A-OKVQA")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/964 [00:00<?, ?B/s]

(…)-00000-of-00002-c1d24de3bacb5e0c.parquet:   0%|          | 0.00/462M [00:00<?, ?B/s]

(…)-00001-of-00002-6b4f3abe2dc385d0.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

(…)-00000-of-00001-b2bd0de231b6326a.parquet:   0%|          | 0.00/60.4M [00:00<?, ?B/s]

(…)-00000-of-00001-d306bf3ad53b6618.parquet:   0%|          | 0.00/337M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17056 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1145 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6702 [00:00<?, ? examples/s]

In [5]:
import torch
from difflib import SequenceMatcher
from transformers import AutoProcessor, AutoModelForCausalLM
from huggingface_hub import hf_hub_download
from PIL import Image

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the processor and model, and move the model to GPU
processor = AutoProcessor.from_pretrained("microsoft/git-base-textvqa")
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-textvqa").to(device)

# file_path = hf_hub_download(repo_id="nielsr/textvqa-sample", filename="bus.png", repo_type="dataset")
flag = 0
num = 10

for i in range(num):
    word_list = ds["validation"][i]['choices']
    image = ds["validation"][i]['image']
    question = ds["validation"][i]['question']
    answer = ds["validation"][i]['choices'][ds["validation"][i]['correct_choice_idx']]

    # Preprocess the image and move pixel values to GPU
    pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)

    # Process the question and move input_ids to GPU
    input_ids = processor(text=question, add_special_tokens=False).input_ids
    input_ids = [processor.tokenizer.cls_token_id] + input_ids
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)

    # Generate the answer using the model on GPU
    generated_ids = model.generate(pixel_values=pixel_values, input_ids=input_ids, max_length=50)

    # Decode the generated text
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
    generated_word = generated_text[0].split()[-1]

    # Find the best matching word using SequenceMatcher
    best_word = max(word_list, key=lambda w: SequenceMatcher(None, generated_word, w).ratio())

    # Check if the best word matches the correct answer
    if best_word == answer:
        flag += 1

    #print(best_word, answer)

# Calculate the accuracy
accuracy = flag / num
print(f"Accuracy: {accuracy * 100:.2f}%")


preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

Accuracy: 40.00%


In [6]:
from transformers import ViltProcessor, ViltForQuestionAnswering
from difflib import SequenceMatcher
import torch
from PIL import Image

# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# prepare image + question
flag = 0
num = 200

for i in range(num):
    word_list = ds["validation"][i]['choices']
    image = ds["validation"][i]['image']
    text = ds["validation"][i]['question']
    answer = ds["validation"][i]['choices'][ds["validation"][i]['correct_choice_idx']]

    # Load processor and model
    processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
    model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").to(device)  # Move model to GPU

    # Prepare inputs
    encoding = processor(image, text, return_tensors="pt").to(device)  # Move inputs to GPU

    # Forward pass (inference)
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    generated_word = model.config.id2label[idx]

    # Use SequenceMatcher to find the best word
    best_word = max(word_list, key=lambda w: SequenceMatcher(None, generated_word, w).ratio())

    # Check if the best word matches the correct answer
    if best_word == answer:
        flag += 1

# Calculate the accuracy
accuracy = flag / num
print(f"Accuracy: {accuracy * 100:.2f}%")


preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/136k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

Accuracy: 44.00%
